# Install Whisper

In [1]:
# !apt install -y ffmpeg

# Get WhisperTimeSync

In [2]:
# !rm -rf WhisperTimeSync
# !git clone https://github.com/EtienneAb3d/WhisperTimeSync.git

# Transcribe

In [3]:
import json
from get_torah_text_using_sefaria import get_chapter_string
import scipy.io.wavfile as wavfile
import io
from six.moves.urllib.request import urlopen
import pathlib
import soundfile as sf
from tqdm import tqdm
from nikud_and_teamim import remove_nikud, replace_teamim_with_emphasis, remove_nikud_and_teamim


!nvidia-smi


Sun Jul  7 08:51:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:01:00.0 Off |                  Off |
| 42%   59C    P8              23W / 300W |  21282MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

we want to adapt the text to time, in iterations.
1) Ketiv Maleh (בֹּקֶר -> בוקר) - becuse we use hebrew model that trained on modern hebrew in without nikud in Ketiv Male this is the first step to adapt the text to the model.
2) Ketiv Haser without teamim - it closer to our data.
3) Ketiv Haser with teamim - now it is much simpler to adapt the text to the model.

In [4]:
# get the text
with open("text.txt", "r") as f:
    original_text = f.read() 

# the first steps are ketiv maleh, for that we use the API of Dicta.
from remove_nikud_dicta import remove_nikud_dicta
ketiv_maleh = remove_nikud_dicta(original_text) # step 2

# before the step of ketiv maleh we want another step of text without "׃" or "־".
cleaned_ketiv_maleh = ketiv_maleh.replace("־", " ").replace("׃", ".") # step 1

# the second step is the ketiv haser without teamim, for that we use our own library
from nikud_and_teamim import remove_nikud_and_teamim, remove_nikud
ketiv_haser_no_teamim = remove_nikud_and_teamim(original_text) # step 3

# the third step is the ketiv haser with teamim, for that we use our own library
ketiv_haser_teamim = remove_nikud(original_text) # step 4


In [5]:
print(step_3_text)

NameError: name 'step_3_text' is not defined

In [32]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import librosa
import srt
from datetime import timedelta
from transformers import pipeline
import os
import soundfile as sf
import scipy.signal
from vad import EnergyVAD  # Make sure you have installed the energy_vad package
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned model and processor
# model = WhisperForConditionalGeneration.from_pretrained("ivrit-ai/whisper-v2-pd1-e1").to(device)
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2").to(device)
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")

model.generation_config.language = "he"

# Define maximum audio segment length
MAX_SEGMENT_LENGTH = 30 * 1000  # 30 seconds in milliseconds


# Function to create SRT file from audio segment and starting time
def create_srt_segment(result, start_time=0):
    """
    Creates an SRT formatted string from the Whisper result.

    Args:
        result: The output from the Whisper pipeline.
        start_time: The starting time (in seconds) for this segment within the full audio.

    Returns:
        An SRT formatted string for the given segment.
    """
    subtitles = []
    current_line = ""
    segment_start_time = None

    for i, segment in enumerate(result["chunks"]):
        word = segment["text"].strip()
        if segment_start_time is None:
            segment_start_time = segment["timestamp"][0]

        if len(current_line + word) > 20:
            end_time = segment["timestamp"][0] + start_time
            subtitles.append(srt.Subtitle(index=len(subtitles) + 1,
                                          start=timedelta(seconds=segment_start_time + start_time),
                                          end=timedelta(seconds=end_time),
                                          content=current_line.strip()))
            current_line = word + " "
            segment_start_time = segment["timestamp"][0]
        else:
            current_line += word + " "

    # Add the last subtitle if the audio segment time is less than 0.1 seconds
    if current_line:
        subtitles.append(srt.Subtitle(index=len(subtitles) + 1,
                                      start=timedelta(seconds=segment_start_time + start_time),
                                      end=timedelta(seconds=result["chunks"][-1]["timestamp"][1] + start_time),
                                      content=current_line.strip()))

    return srt.compose(subtitles)


# Function to process an audio segment
def process_audio_segment(audio_segment, start_time=0):
    """
    Processes a single audio segment to generate the SRT content.

    Args:
        audio_segment: The audio data for this segment.
        start_time: The starting time (in seconds) for this segment within the full audio.

    Returns:
        An SRT formatted string for the given segment.
    """
    asr = pipeline("automatic-speech-recognition", model=model, tokenizer=processor.tokenizer,
                   feature_extractor=processor.feature_extractor, device_map="auto")

    result = asr(audio_segment, return_timestamps="word",
                        generate_kwargs = {"language":"<|he|>",
                                           "task": "transcribe"})
    
    srt_content = create_srt_segment(result, start_time)
    return srt_content


# Function to split audio file into segments
def split_audio(audio_file, output_dir):
    """
    Splits the audio file into segments based on silence detection.

    Args:
        audio_file: Path to the input audio file.
        output_dir: Directory to save the temporary audio segments.

    Returns:
        A list of file names for the created audio segments.
    """
    FRAME_LENGTH = 20  # in milliseconds
    vad = EnergyVAD(
        sample_rate=16000,
        frame_length=FRAME_LENGTH,
        frame_shift=FRAME_LENGTH,
        energy_threshold=0.002,
        pre_emphasis=0.95,
    )

    audio, sr = librosa.load(audio_file, sr=16000)
    voice_activity = vad(audio)

    # Apply median filter to smooth the voice activity detection
    voice_activity_median = scipy.signal.medfilt(voice_activity, kernel_size=15)


    segments = []
    segment_files = []
    start = 0
    total_frames = len(voice_activity_median)

    base_filename = os.path.splitext(os.path.basename(audio_file))[0]

    while start < total_frames:
        for end in range(min(start + MAX_SEGMENT_LENGTH // FRAME_LENGTH - 1, total_frames), start, -1):
            if end >= len(voice_activity_median):
                end = len(voice_activity_median) - 1
            if not voice_activity_median[end]:
                break

        segment = audio[start * FRAME_LENGTH * sr // 1000:end * FRAME_LENGTH * sr // 1000]
        segments.append(segment)

        segment_filename = f"{base_filename}_{len(segments):03d}.wav"
        segment_path = os.path.join(output_dir, segment_filename)
        sf.write(segment_path, segment, sr)
        segment_files.append(segment_filename)

        start = end + 1
    # if the last one is too short (less than 0.1 seconds), we throw it away
    if len(segments) > 1:
        last_segment_duration = len(segments[-1]) / sr
        if last_segment_duration < 0.1:
            os.remove(os.path.join(output_dir, segment_files[-1]))
            segment_files.pop()
            segments.pop()
    
    return segment_files
    


# Main function to run the process
def generate_srt_from_audio(audio_file, output_srt_file):
    """
    Generates an SRT file from an audio file.

    Args:
        audio_file: Path to the input audio file.
        output_srt_file: Path to the output SRT file.
    """
    output_dir = "temp_audio_segments"
    os.makedirs(output_dir, exist_ok=True)

    segment_files = split_audio(audio_file, output_dir)

    full_srt_content = ""
    total_duration = 0

    for i, segment_file in enumerate(tqdm(segment_files)):
        segment_path = os.path.join(output_dir, segment_file)
        audio_segment, _ = librosa.load(segment_path, sr=16000)
        srt_segment = process_audio_segment(audio_segment, total_duration)
        full_srt_content += srt_segment + "\n"
        total_duration += librosa.get_duration(y=audio_segment, sr=16000) # in seconds
    
    # if the last one is too short (less than 0.2 seconds), we merge it with the previous one if total duration is less than MAX_SEGMENT_LENGTH
    # if total_duration < MAX_SEGMENT_LENGTH / 1000:
    #     last_subtitle = full_srt_content.split("\n")[-2]
    #     if last_subtitle:
    #         last_subtitle = srt.parse(last_subtitle)
    #         if last_subtitle and (last_subtitle[-1].end.total_seconds() - last_subtitle[-1].start.total_seconds()) < 0.2:
    #             last_subtitle[-2].end = last_subtitle[-1].end
    #             last_subtitle.pop()
    #             full_srt_content = srt.compose(last_subtitle)
    
    with open(output_srt_file, "w", encoding="utf-8") as f:
            f.write(full_srt_content)

    # Remove temporary audio segments
    for segment_file in segment_files:
        os.remove(os.path.join(output_dir, segment_file))
    os.rmdir(output_dir)


dir = "/app/download and process the data/slice long data/automatic using WhisperTimeSync/"
# Call the main function
generate_srt_from_audio(dir + "audio.wav", dir + "output.srt")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
 91%|█████████▏| 64/70 [01:12<00:06,  1.13s/it]


TypeError: unsupported operand type(s) for +: 'NoneType' and 'float'

In [1]:
!java -Xmx2G -jar WhisperTimeSync/distrib/WhisperTimeSync.jar "output.srt" "text.txt" he



Output (text.txt.srt):


In [ ]:
len(audio)

32644576

# the method that we used before:

In [ ]:
# WITH_TEAMIM = False
# WITH_STRESS = True
# SR = 16000


# if WITH_TEAMIM and WITH_STRESS:
#     print("WITH_TEAMIM and WITH_STRESS cannot be True at the same time.")
#     exit(1)


In [ ]:
# Example with the first chapter of Genesis:
with open('links_for_audio_from_929.json', 'r') as f:
    links_for_audio_from_929 = json.load(f)

url = links_for_audio_from_929["books"][0]["chapters"][0]["link"]

# Download the audio file
z = io.BytesIO(urlopen(url).read())
pathlib.Path((f"{book_name}_{chapter}.mp3")).write_bytes(z.getbuffer())
audio, sr = librosa.load(f"{book_name}_{chapter}.mp3", sr=SR)


# Get the text from Sefaria
book_name = "Genesis"
chapter = 1
text = get_chapter_string(book_name, chapter)

# remove the nikud and replace the teamim with empsis
if WITH_TEAMIM:
    text = remove_nikud(text)
else:
    if WITH_STRESS:
        text = remove_nikud(text)
        text = replace_teamim_with_emphasis(text)
    else:
        text = remove_nikud_and_teamim(text)
text = "בראשֽית " + "פרק " + "אלף " + text

with open(f"{book_name}_{chapter}.txt", "w") as f:
    f.write(text)

# Transcribe the audio to get the timestamps (as srt file)
!python3 WhisperTimeSync/transcribe.py /content/"{book_name}_{chapter}.mp3" large-v3

# Align(sync) the text with the audio
!java -Xmx2G -jar WhisperTimeSync/distrib/WhisperTimeSync.jar "{book_name}_{chapter}.srt" "{book_name}_{chapter}.txt" he

# Load the srt file
with open(f"{book_name}_{chapter}.txt.srt", "r") as f:
    srt = f.read()

dataset = {"text": [], "start": [], "end": [], "audio_file": []}
# Add the times, text, and audio file name(save each sentence as a separate audio file), to the dataset
for i, line in enumerate(srt.split("\n")):
    if i % 4 == 0:
        dataset["start"].append(line)
    elif i % 4 == 1:
        dataset["end"].append(line)
    elif i % 4 == 2:
        dataset["text"].append(line)
    elif i % 4 == 3:
        dataset["audio_file"].append(f"{book_name}_{chapter}_{i//4}.wav")
        start = int(float(dataset["start"][-1].replace(",", ".").replace(" --> ", "")) * SR)
        end = int(float(dataset["end"][-1].replace(",", ".").replace(" --> ", "")) * SR)
        sf.write(f"{book_name}_{chapter}_{i//4}.wav", audio[start:end], SR)


FileNotFoundError: [Errno 2] No such file or directory: 'links_for_audio_from_929.json'

In [ ]:
import librosa
import requests
import json
from get_torah_text_using_sefaria import get_chapter_string
import scipy.io.wavfile as wavfile
import io
from six.moves.urllib.request import urlopen
import pathlib
import soundfile as sf
from tqdm import tqdm
SR = 16000


books_data = [
    {"name": "Bereshit", "number": 1, "chapters": 50, "sefaria_name": "Genesis"},
    {"name": "Shemot", "number": 2, "chapters": 40, "sefaria_name": "Exodus"},
    {"name": "Vaikra", "number": 3, "chapters": 27, "sefaria_name": "Leviticus"},
    {"name": "Bamidbar", "number": 4, "chapters": 36, "sefaria_name": "Numbers"},
    {"name": "Dvarim", "number": 5, "chapters": 34, "sefaria_name": "Deuteronomy"}
]
sefer_names = ["", "בְּרֵאשִֽׁית", "שְׁמֽות", "וַיִּקְרָֽא", "בַּמִּדְבָּֽר", "דְּבָרִֽים"]
otiyot_gematria = ["", "אָֽלֶף", "בֵּֽית", "גִּֽימֶל", "דָּֽלֶת", "הֵֽא", "וָֽו", "זַֽיִן", "חֵֽית", "טֵֽית", "יֽוֹד", "יֽוֹד-אָֽלֶף", "יֽוֹד-בֵּֽית", "יֽוֹד-גִּֽימֶל", "יֽוֹד-דָּֽלֶת", "טֵֽית-וָֽו", "טֵֽית-זַֽיִן", "יֽ-זַֽיִן", "יֽוֹד-חֵֽית", "יֽוֹד-טֵֽית", "כַּֽף", "כַּֽף-אָֽלֶף", "כַּֽף-בֵּֽית", "כַּֽף-גִּֽימֶל", "כַּֽף-דָּֽלֶת", "כַּֽף-הֵֽא", "כַּֽף-וָֽו", "כַּֽף-זַֽיִן", "כַּֽף-חֵֽית", "כַּֽף-טֵֽית", "לָֽמֶד", "לָֽמֶד-אָֽלֶף", "לָֽמֶד-בֵּֽית", "לָֽמֶד-גִּֽימֶל", "לָֽמֶד-דָּֽלֶת", "לָֽמֶד-הֵֽא", "לָֽמֶד-וָֽו", "לָֽמֶד-זַֽיִן", "לָֽמֶד-חֵֽית", "לָֽמֶד-טֵֽית", "מֵֽם", "מֵֽם-אָֽלֶף", "מֵֽם-בֵּֽית", "מֵֽם-גִּֽימֶל", "מֵֽם-דָּֽלֶת", "מֵֽם-הֵֽא", "מֵֽם-וָֽו", "מֵֽם-זַֽיִן", "מֵֽם-חֵֽית", "מֵֽם-טֵֽית", "נֽוּן", "נֽוּן-אָֽלֶף", "נֽוּן-בֵּֽית", "נֽוּן-גִּֽימֶל", "נֽוּן-דָּֽלֶת", "נֽוּן-הֵֽא", "נֽוּן-וָֽו", "נֽוּן-זַֽיִן", "נֽוּן-חֵֽית", "נֽוּן-טֵֽית", "סָֽמֶךְ"]
otiyot_gematria = [remove_nikud(ot) for ot in otiyot_gematria]
dataset = {"text": [], "audio_file": []}

with open('links_for_audio_from_929.json', 'r') as f:
    links_for_audio_from_929 = json.load(f)

# Calculate total number of chapters
total_chapters = sum(book["chapters"] for book in books_data)

# Create a progress bar
pbar = tqdm(total=total_chapters)

# Loop over all the chapters of all the books
for book in books_data:
    book_name = book["name"]
    sefaria_name = book["sefaria_name"]
    for chapter in range(1, book["chapters"]+1):
        if chapter % 27 != 0: # test on small data
            continue
        url = links_for_audio_from_929["books"][book["number"]-1]["chapters"][chapter-1]["link"]
        z = io.BytesIO(urlopen(url).read())
        pathlib.Path((f"{book_name}_{chapter}.mp3")).write_bytes(z.getbuffer())
        audio, sr = librosa.load(f"{book_name}_{chapter}.mp3", sr=SR)
        text = get_chapter_string(sefaria_name, chapter)
        if WITH_TEAMIM:
            text = remove_nikud(text)
        else:
            if WITH_STRESS:
                text = remove_nikud(text)
                text = replace_teamim_with_emphasis(text)
            else:
                text = remove_nikud_and_teamim(text)
        text = f"{sefer_names[book['number']]} " + f"פרק {otiyot_gematria[chapter]} " + text
        with open(f"{book_name}_{chapter}.txt", "w") as f:
            f.write(text)
        !python3 WhisperTimeSync/transcribe.py /content/"{book_name}_{chapter}.mp3" large-v3
        !java -Xmx2G -jar WhisperTimeSync/distrib/WhisperTimeSync.jar "{book_name}_{chapter}.srt" "{book_name}_{chapter}.txt" he
        with open(f"{book_name}_{chapter}.txt.srt", "r") as f:
            srt = f.read()

        for i, line in enumerate(srt.split("\n")):
            if i % 4 == 0: # The line is a number of the subtitle
                continue
            if i % 4 == 1: # The line is a time
                start, end = line.split(" --> ")
                start = start.replace(",", ".")
                end = end.replace(",", ".")
                start = start.split(":")
                end = end.split(":")
                start = float(start[0])*3600 + float(start[1])*60 + float(start[2])
                end = float(end[0])*3600 + float(end[1])*60 + float(end[2])
                start = float(start)
                end = float(end)
                continue
            if i % 4 == 2: # The line is the text
                dataset["text"].append(line)
                dataset["audio_file"].append(f"{book_name}_{chapter}_{int(i/4):02}.wav")
                # Save the audio file
                sf.write(f"{book_name}_{chapter}_{int(i/4):02}.wav", audio[int(start*SR):int(end*SR)], SR)
            if i % 4 == 3: # The line is an empty line
                continue
        # Remove the original audio file and the srt file
        !rm "{book_name}_{chapter}.mp3"
        !rm "{book_name}_{chapter}.txt"
        !rm "{book_name}_{chapter}.txt.srt"
        !rm "{book_name}_{chapter}.srt"

        # Update the progress bar
        pbar.update()

# Save the dataset
with open("dataset.json", "w") as f:
    json.dump(dataset, f)
print("Done")


  4%|▎         | 7/187 [33:04<14:10:21, 283.45s/it]


Loading Whisper model: large-v3 ...
Transcribing: /content/Bereshit_27.mp3 ...
 בראשית פרק קפזיים תודה רבה. תודה רבה. תודה רבה. תודה רבה. תודה רבה. תודה רבה. תודה רבה. תודה רבה. תודה רבה. תודה רבה. תודה רבה. תודה רבה.
Saving: /content/Bereshit_27.mp3.srt ...

Output (Bereshit_27.txt.srt):



  1%|          | 1/187 [00:55<2:53:22, 55.93s/it]

Loading Whisper model: large-v3 ...
Transcribing: /content/Shemot_27.mp3 ...
 שמות פרק קפזיין תעשה נחושת ועשית לו מחבר מעשה רשת נחושת ועשית על הרשת ארבע תבעות נחושת על ארבע קצותיו ונתתה אותה תחת קרקוב המזבח מלמטה והייתה הרשת עד חצי המזבח ועשית ודים למזבח בדי עצי שיטים וציפיתה אותם נחושת והובה את בדיו בתבעות והיו הבדים על שתי צלעות המזבח ביסת אותו נבוב לוחות תעשה אותו כאשר הראה אותך בהר, כן יעשו ועשית את חצר המשכן לפעת נגב תימנה כלעים לחצר שש מושזר מאה ועמה אורך לפאה האחת ועמודיו עשרים ועדניהם עשרים נחושת ווי העמודים וחשוקיהם כסף וכן לפעת צפון באורך כלעים מאה אורך ועמודיו עשרים ועדניהם עשרים נחושת ווי העמודים וחשוקיהם כסף ורוחב החצר לפעת ים כלעים חמישים עמה עמודיהם עשרה ועדניהם עשרה ורוחב החצר לפעת קדמה מזרחה חמישים עמה וחמש עשרה עמה כלעים לקטף עמודיהם שלושה ועדניהם שלושה ולקטף השנית חמש עשרה כלעים עמודיהם שלושה ועדניהם שלושה אולשר החצר מסך עשרים עמה תחלת וAll fire ותולה עדשני ושש מושזר מעשי רוקם עמודיהם ארבעה ועדניהם ארבעה שבא� obed besond One כל עמודי החצר סביב לאדנה אלrill겠 impossibl


  1%|          | 2/187 [03:37<6:03:37, 117.93s/it]

...

Loading Whisper model: large-v3 ...
Transcribing: /content/Bamidbar_27.mp3 ...
 במדבר פרק קפזיין והוא לא היה בתוך העדה הנועדים על אדוני בעדת קורח כי וחת אומת ובנים לא היו לו למה ייגרה השם אבינו מתוך משפחתו כי אין לו בן תנה לנו אחוזה בתוך אחרי אבינו ויקרב משה את משפטן לפני אדוני ויאמר אדוני אל משה לאמור כן בנות סלופחת דוברות נתון תיתן להם אחוזת נחת ונחלה בתוך אחרי אביהם והעברתה את נחלת אביהן להן ואל בני ישראל תדבר לאמור איש כי ימות ובן אין לו והעברתם את נחלתו לביתו ואם אין לו בת אונטתם את נחלתו לאיחיו ואם אין לו אחים אונטתם את נחלתו לאחרי אביו ואם אין אחים לאביו אונטתם את נחלתו לשארו הקרוב אליו ממיםו וירש אותה, והייתה לבני ישראל לחוקת משפט, כאשר ציווה אדוני את משה. ויאמר אדוני אל משה, עלי אל הר העברים הזה, וראה את הארץ, אשר נתתי לבני ישראל. וראית אותה, ונאספת אל עמך גם אתה, כאשר נאסף אהרון אחיך. כאשר מריתם פי במדבר צין, במריבת העדה, להקדישני ומים לעיניהם, הם מי מריבת קדש, מדבר צין. וידבר משה אל אדוני לאמור, הפקוד אדוני אלוהי הרוחות לכל בשר, איש על העדה, אשר יצא לפניהם, ואשר יבוא לפניהם,


  2%|▏         | 4/187 [09:17<7:36:22, 149.63s/it]

Loading Whisper model: large-v3 ...
Transcribing: /content/Dvarim_27.mp3 ...
 דברים, פרק קפזיין תודה רבה תודה רבה תודה רבה תודה רבה תודה רבה תודה רבה תודה רבה
Saving: /content/Dvarim_27.mp3.srt ...

Output (Dvarim_27.txt.srt):



  3%|▎         | 5/187 [10:11<5:48:45, 114.98s/it]

Done


In [ ]:

book_name = "Bereshit"
chapter = 27
url = links_for_audio_from_929["books"][1]["chapters"][chapter-1]["link"]
z = io.BytesIO(urlopen(url).read())
pathlib.Path((f"{book_name}_{chapter}.mp3")).write_bytes(z.getbuffer())
!python3 WhisperTimeSync/transcribe.py /content/"{book_name}_{chapter}.mp3" large-v3

Loading Whisper model: large-v3 ...


In [ ]:
# Save the dataset
with open("dataset.json", "w", encoding="utf-8") as f:
    json.dump(dataset, f)

In [ ]:
# play random audio and print its text
import random
import IPython.display as ipd
index = random.randint(0, len(dataset["audio_file"]))
print(dataset["text"][index])
ipd.Audio(dataset["audio_file"][index])


הבדֽים על־שתֽי צלעֽת המזבֽח בשאֽת


In [ ]:
index = 14
print(dataset["text"][index])
ipd.Audio(dataset["audio_file"][index])

שְׁמֽות פרק כֽף-זֽין ועשֽית


In [ ]:
# Sync the times of the real text using the srt file
!java -Xmx2G -jar WhisperTimeSync/distrib/WhisperTimeSync.jar "{book_name}_{chapter}.srt" "{book_name}_{chapter}.txt" he


Output (Genesis_1.txt.srt):


In [ ]:
books_data = [
    {"name": "Bereshit", "number": 1, "chapters": 50, "sefaria_name": "Genesis"},
    {"name": "Shemot", "number": 2, "chapters": 40, "sefaria_name": "Exodus"},
    {"name": "Vaikra", "number": 3, "chapters": 27, "sefaria_name": "Leviticus"},
    {"name": "Bamidbar", "number": 4, "chapters": 36, "sefaria_name": "Numbers"},
    {"name": "Dvarim", "number": 5, "chapters": 34, "sefaria_name": "Deuteronomy"}
]
sefer_names = ["", "בְּרֵאשִֽׁית", "שְׁמֽות", "וַיִּקְרָֽא", "בַּמִּדְבָּֽר", "דְּבָרִֽים"]
otiyot_gematria = ["", "אָֽלֶף", "בֵּֽית", "גִּֽימֶל", "דָּֽלֶת", "הֵֽא", "וָֽו", "זַֽיִן", "חֵֽית", "טֵֽית", "יֽוֹד", "יֽוֹד-אָֽלֶף", "יֽוֹד-בֵּֽית", "יֽוֹד-גִּֽימֶל", "יֽוֹד-דָּֽלֶת", "טֵֽית-וָֽו", "טֵֽית-זַֽיִן", "יֽ-זַֽיִן", "יֽוֹד-חֵֽית", "יֽוֹד-טֵֽית", "כַּֽף", "כַּֽף-אָֽלֶף", "כַּֽף-בֵּֽית", "כַּֽף-גִּֽימֶל", "כַּֽף-דָּֽלֶת", "כַּֽף-הֵֽא", "כַּֽף-וָֽו", "כַּֽף-זַֽיִן", "כַּֽף-חֵֽית", "כַּֽף-טֵֽית", "לָֽמֶד", "לָֽמֶד-אָֽלֶף", "לָֽמֶד-בֵּֽית", "לָֽמֶד-גִּֽימֶל", "לָֽמֶד-דָּֽלֶת", "לָֽמֶד-הֵֽא", "לָֽמֶד-וָֽו", "לָֽמֶד-זַֽיִן", "לָֽמֶד-חֵֽית", "לָֽמֶד-טֵֽית", "מֵֽם", "מֵֽם-אָֽלֶף", "מֵֽם-בֵּֽית", "מֵֽם-גִּֽימֶל", "מֵֽם-דָּֽלֶת", "מֵֽם-הֵֽא", "מֵֽם-וָֽו", "מֵֽם-זַֽיִן", "מֵֽם-חֵֽית", "מֵֽם-טֵֽית", "נֽוּן", "נֽוּן-אָֽלֶף", "נֽוּן-בֵּֽית", "נֽוּן-גִּֽימֶל", "נֽוּן-דָּֽלֶת", "נֽוּן-הֵֽא", "נֽוּן-וָֽו", "נֽוּן-זַֽיִן", "נֽוּן-חֵֽית", "נֽוּן-טֵֽית", "סָֽמֶךְ"]
otiyot_gematria = [remove_nikud(ot) for ot in otiyot_gematria]


# Load the data of links_for_audio_from_929.json

with open('WhisperTimeSync/links_for_audio_from_929.json', 'r') as f:
    links_for_audio_from_929 = json.load(f)


import requests
url = links_for_audio_from_929["books"][0]["chapters"][0]["link"]
# Download (to RAM) the audio file for the first chapter of Bereshit
r = requests.get(url, allow_redirects=True)


# for book in books_data:
#     print(book['name'])
#     for chapter in range(1, book['chapters'] + 1):
#         print(chapter)
#         print(links_for_audio_from_929[book['sefaria_name']][str(chapter)])


#         להוסיף את ההורדה של האודיו ומשיכת הטקסט הרלוונטי
#         ואז להפעיל את הקוד של וויספרטיימסינק על הטקסט והאודיו







In [ ]:
!cat /content/51.Shmot_1.srt

# Synchronize

In [ ]:
!java -Xmx2G -jar WhisperTimeSync/distrib/WhisperTimeSync.jar /content/51.Shmot_1.srt /content/Exodus.1.txt he

In [ ]:
!cat /content/Exodus.1.txt.srt